In [2]:
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()

data_folder = os.getenv("DATA_FOLDER")

#load the three relevant dataframes
claim_df = pd.read_csv(f'{data_folder}/articles_with_claims.csv', sep=";",index_col=0)
search_result_df = pd.read_csv(f'{data_folder}/All_Search_Results_Combined.csv', index_col=0)
search_result_df.drop(columns="X", inplace=True)
fm_articles_df = pd.read_excel(f'{data_folder}/FM_Articles_with_ids_processed.xlsx', index_col=0)

In [3]:
claim_df.rename(columns={'url': 'URL', 'title': 'Title', 'body': 'Body', 'claims': 'Claims'}, inplace=True) #rename columns to make them consistent between dataframes, enabling the flexibility of functions

Claims were extracted for 1769 articles. Note, however, that these are stored as strings, which can result in potential evaluation errors, making them unusuable.

In [4]:
claim_df

,URL,Title,Body,id_body,Claims,errors,output_errors,pre_processing_errors
1,https://abc13.com/us-shortages-gas-shortage-20...,"US shortages 2021: Gas, lumber prices soar; Ke...","WATCH LIVE NEW YORK -- Chicken, lumber, microc...","{'headline': 'US shortages 2021: Gas, lumber p...","{\n""article_title"": ""US shortages 2021: Gas, l...",False,False,False
2,https://abc7.com/covid-supply-chain-shortage-2...,Global supply chain problems now leading to em...,WATCH LIVE LOS ANGELES (KABC) -- At the beginn...,{'headline': 'Global supply chain problems now...,"{\n""article_title"": ""Global supply chain probl...",False,False,False
3,https://abcnews.go.com/Politics/biden-replace-...,Biden to replace White House doctor with long-...,O'Connor will take on a role that faced scruti...,{'headline': 'Biden to replace White House doc...,"{\n ""article_title"": ""Biden to replace White H...",False,False,False
4,https://abcnews.go.com/Politics/whats-causing-...,What’s causing America’s massive supply-chain ...,Untangling supply chain woes could take much l...,{'headline': 'What’s causing America’s massive...,"{\n""article_title"": ""What’s causing America’s ...",False,False,False
5,https://abcnews.go.com/US/nature-based-man-mad...,Nature-based or lab leak? Unraveling the debat...,Accomplished scientists and public health offi...,{'headline': 'Nature-based or lab leak? Unrave...,"{\n""article_title"": ""Nature-based or lab leak?...",False,False,False
...,...,...,...,...,...,...,...,...
1765,https://www.washingtonpost.com/politics/trump-...,Trump clings to one marker as a sign of succes...,clockThis article was published more than 3 ye...,{'headline': 'Trump clings to one marker as a ...,"{\n""article_title"": ""Trump clings to one marke...",False,False,False
1766,https://www.washingtonpost.com/politics/trump-...,Trump says ‘there was no reason’ for officer t...,clockThis article was published more than 3 ye...,{'headline': 'Trump says ‘there was no reason’...,"{\n""article_title"": ""Trump says ‘there was no ...",False,False,False
1767,https://www.washingtonpost.com/politics/trump-...,Trump Organization removes indicted top financ...,clockThis article was published more than 3 ye...,{'headline': 'Trump Organization removes indic...,"{\n""article_title"": ""Trump Organization remove...",False,False,False
1768,https://www.washingtonpost.com/politics/trump-...,Trump business and its longtime chief financia...,clockThis article was published more than 3 ye...,{'headline': 'Trump business and its longtime ...,"{\n ""article_title"": ""Trump business and its l...",True,False,False


We only care about the search queries that relate to the Fake or Misleading articles

In [5]:
search_result_df = search_result_df[search_result_df['URL'].isin(fm_articles_df['URL'])] #filter down to only contain results that have F/M articles as URL
search_result_df.reset_index(drop=True, inplace=True) 
search_result_df

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_80,only_rel_85,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls
0,Day_1_1,R_10pkUNEtcV6OMCU,Day_1,Coul,0,5,3,35,1,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,91.055556,"newsweek.com,yahoo.com,azmirror.com,fr24news.c...","100,100,57,92.5,100,100,75,100,95",'https://www.newsweek.com/arizona-state-senato...
1,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,95.277778,"abc15.com,salon.com,cnn.com,politico.com,cbsne...","100,87.5,87.5,100,95,95,100,100,92.5",'https://www.abc15.com/news/state/poll-many-re...
2,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,83.450000,"cnn.com,thehill.com,salon.com,washingtontimes....","87.5,80,87.5,42,87.5,100,92.5,87.5,85,85",'https://www.cnn.com/2021/07/18/politics/fact-...
3,Day_1_1,R_3KT6q7Vntwvmg8Z,Day_1,True,1,7,3,33,1,FM,...,0.0,0.0,0.0,0.0,0.0,0.0,92.812500,"thehill.com,recorder.maricopa.gov,washingtonpo...","80,100,100,100,100,92.5,75,95",'https://thehill.com/homenews/campaign/563100-...
4,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,96.000000,"apnews.com,detroitnews.com,cnn.com,apnews.com,...","95,92.5,87.5,95,95,100,100,100,100,95",'https://apnews.com/article/technology-joe-bid...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,Day_9_3,R_1P6MQAwGMgs0Loo,Day_9,True,1,5,3,39,0,FM,...,0.0,0.0,0.0,0.0,0.0,0.0,94.166667,"googleadservices.com,scientificamerican.com,wh...","100,77.5,87.5,100,100,100","'https://www.googleadservices.com/','https://w..."
1263,Day_9_3,R_rcgF4h6LylUXPUd,Day_9,Misl,0,1,2,23,0,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,92.500000,"covid19.nih.gov,grants.nih.gov,grants.nih.gov,...","100,87.5,100,87.5,87.5",'https://covid19.nih.gov/funding#:~:text=NIH%2...
1264,Day_9_3,R_3JqGYdN07AFVzOf,Day_9,Misl,0,1,1,41,0,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,85.777778,"science.org,the-scientist.com,rollcall.com,van...","100,100,75,82.5,82.5,95,80,57,100",'https://www.science.org/content/article/nih-s...
1265,Day_9_3,R_SHIu8anABwVwNMZ,Day_9,True,1,7,4,33,1,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,7.500000,"',zerohedge.com,madisonarealymesupportgroup.co...",7.5,'/advanced_search?q=https://www.zerohedge.com/...


As shown below, a noticeable portion of the Aslett et al. data does not contain any clean urls at all, making them unusuable for our purposes. This brings the potential number of valid search queries down from 1267 to 961, a decrease of 24.15%.

In [6]:
search_result_df[search_result_df['all_clean_urls'].isna()]

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_80,only_rel_85,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls
6,Day_1_1,R_3LXaF7Z7z740d2k,Day_1,Misl,0,1,1,26,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Day_1_1,R_3LXaF7Z7z740d2k,Day_1,Misl,0,1,1,26,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Day_1_1,R_2PpsctGRoIizGAS,Day_1,Misl,0,1,1,39,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Day_1_1,R_2PpsctGRoIizGAS,Day_1,Misl,0,1,1,39,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Day_1_1,R_2PpsctGRoIizGAS,Day_1,Misl,0,1,1,39,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,Day_9_3,R_2QEWqk6A2nnsg7O,Day_9,True,1,6,4,26,0,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1255,Day_9_3,R_1mQbS6eXGog8n3j,Day_9,Coul,0,4,3,57,1,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1257,Day_9_3,R_2QEWqk6A2nnsg7O,Day_9,True,1,6,4,26,0,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1259,Day_9_3,R_2QEWqk6A2nnsg7O,Day_9,True,1,6,4,26,0,FM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
search_result_df = search_result_df[search_result_df['all_clean_urls'].notna()]
search_result_df.reset_index(drop=True, inplace=True) 
search_result_df

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_80,only_rel_85,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls
0,Day_1_1,R_10pkUNEtcV6OMCU,Day_1,Coul,0,5,3,35,1,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,91.055556,"newsweek.com,yahoo.com,azmirror.com,fr24news.c...","100,100,57,92.5,100,100,75,100,95",'https://www.newsweek.com/arizona-state-senato...
1,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,95.277778,"abc15.com,salon.com,cnn.com,politico.com,cbsne...","100,87.5,87.5,100,95,95,100,100,92.5",'https://www.abc15.com/news/state/poll-many-re...
2,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,83.450000,"cnn.com,thehill.com,salon.com,washingtontimes....","87.5,80,87.5,42,87.5,100,92.5,87.5,85,85",'https://www.cnn.com/2021/07/18/politics/fact-...
3,Day_1_1,R_3KT6q7Vntwvmg8Z,Day_1,True,1,7,3,33,1,FM,...,0.0,0.0,0.0,0.0,0.0,0.0,92.812500,"thehill.com,recorder.maricopa.gov,washingtonpo...","80,100,100,100,100,92.5,75,95",'https://thehill.com/homenews/campaign/563100-...
4,Day_1_1,R_1Gv8iAs1HEqGuUe,Day_1,Misl,0,1,1,35,1,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,96.000000,"apnews.com,detroitnews.com,cnn.com,apnews.com,...","95,92.5,87.5,95,95,100,100,100,100,95",'https://apnews.com/article/technology-joe-bid...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,Day_9_3,R_3JqGYdN07AFVzOf,Day_9,Misl,0,1,1,41,0,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,98.437500,"factcheck.org,yahoo.com,theintercept.com,polit...","100,100,87.5,100,100,100,100,100",'https://www.factcheck.org/2021/10/scicheck-re...
957,Day_9_3,R_1P6MQAwGMgs0Loo,Day_9,True,1,5,3,39,0,FM,...,0.0,0.0,0.0,0.0,0.0,0.0,94.166667,"googleadservices.com,scientificamerican.com,wh...","100,77.5,87.5,100,100,100","'https://www.googleadservices.com/','https://w..."
958,Day_9_3,R_rcgF4h6LylUXPUd,Day_9,Misl,0,1,2,23,0,FM,...,1.0,1.0,0.0,0.0,0.0,0.0,92.500000,"covid19.nih.gov,grants.nih.gov,grants.nih.gov,...","100,87.5,100,87.5,87.5",'https://covid19.nih.gov/funding#:~:text=NIH%2...
959,Day_9_3,R_3JqGYdN07AFVzOf,Day_9,Misl,0,1,1,41,0,FM,...,0.0,0.0,0.0,1.0,1.0,1.0,85.777778,"science.org,the-scientist.com,rollcall.com,van...","100,100,75,82.5,82.5,95,80,57,100",'https://www.science.org/content/article/nih-s...


The available clean urls are evaluated as tuples

In [8]:
import ast

def safe_literal_eval(input):
    try:
        return ast.literal_eval(input)
    except:
        return ()
    
#search_result_df['URL']
search_result_df['all_clean_urls'] = search_result_df['all_clean_urls'].apply(safe_literal_eval)
search_result_df['all_clean_urls']

C:\Users\danie\AppData\Local\Temp\ipykernel_17200\3465476202.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_result_df['all_clean_urls'] = search_result_df['all_clean_urls'].apply(safe_literal_eval)


0      (https://www.newsweek.com/arizona-state-senato...
1      (https://www.abc15.com/news/state/poll-many-re...
2      (https://www.cnn.com/2021/07/18/politics/fact-...
3      (https://thehill.com/homenews/campaign/563100-...
4      (https://apnews.com/article/technology-joe-bid...
                             ...                        
956    (https://www.factcheck.org/2021/10/scicheck-re...
957    (https://www.googleadservices.com/, https://ww...
958    (https://covid19.nih.gov/funding#:~:text=NIH%2...
959    (https://www.science.org/content/article/nih-s...
960    (/advanced_search?q=https://www.zerohedge.com/...
Name: all_clean_urls, Length: 961, dtype: object

Initilization of log data

In [9]:
#declaring the possible errors for the log file
domain_processing_errors, ng_dict_errors, ng_score_retrieval_errors = 0, 0, 0 #newsguard score related errors

claim_dict_eval_errors = 0 #times a dictionary of claims could not be evaluated (ast.literal_eval)

fm_article_url_not_found = 0
serp_article_not_found = 0 #times a SERP article's could not be retrieved via its url
response_errors = 0 #times an API request failed


claim_retrieval_errors, claim_headline_retrieval_errors = 0, 0 # errors occuring during the retrieval of the claim positions and the headline bool

post_processing_error = 0

serp_dict_empty_error = 0

successful_parse = 0

def reset_error_count():
    global domain_processing_errors, ng_dict_errors, ng_score_retrieval_errors
    global claim_dict_eval_errors
    global fm_article_url_not_found
    global serp_article_not_found
    global response_errors
    global claim_retrieval_errors, claim_headline_retrieval_errors
    global post_processing_error
    global serp_dict_empty_error
    global successful_parse 

    domain_processing_errors, ng_dict_errors, ng_score_retrieval_errors = 0, 0, 0

    claim_dict_eval_errors = 0

    serp_article_not_found = 0
    response_errors = 0 

    claim_retrieval_errors, claim_headline_retrieval_errors = 0, 0

    post_processing_error = 0

    successful_parse = 0

def create_log(file_path):
    global domain_processing_errors, ng_dict_errors, ng_score_retrieval_errors
    global claim_dict_eval_errors
    global fm_article_url_not_found
    global serp_article_not_found
    global response_errors
    global claim_retrieval_errors, claim_headline_retrieval_errors
    global post_processing_error
    global serp_dict_empty_error
    global successful_parse

    error_dict = {'Successful_Parses': successful_parse,
                  'Domain_Processing_Errors': domain_processing_errors,
                  'Newsguard_Dict_Errors': ng_dict_errors,
                  'Newsguard_Score_Retrieval_Errors': ng_score_retrieval_errors,
                  'Claim_Dict_Eval_Errors:': claim_dict_eval_errors,
                  'FM_Article_Retrieval_Errors': fm_article_url_not_found,
                  'SERP_Article_Retrieval_Errors': serp_article_not_found,
                  'SERP_Dict_Empty_Errors': serp_dict_empty_error,
                  'API_Response_Errors': response_errors,
                  'Claim_Position_Retrieval_Errors': claim_retrieval_errors,
                  'Claim_Headline_Retrieval_Errors': claim_headline_retrieval_errors,
                  'Post_Processing_Error': post_processing_error
                  }
    
    with open(file_path, 'w') as f:
        f.write(str(error_dict))

In [10]:

#adds quotes around each domain in order for ast.literal_eval to interpret it properly as a list

def pre_process_domains(domain_string):  
    global domain_processing_errors 
    try:
        return ast.literal_eval(f"'{domain_string}'")
    except:
        print('ERROR: could not convert string of domains to list')
        domain_processing_errors += 1
        return ()

In [11]:
def normalize_quotes(string):
    return string.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")

def get_claim_dict(url, df):
    global claim_dict_eval_errors
    claims = df.loc[df['URL'] == url, 'Claims']
    if not claims.empty:
        specific_string_content = claims.iloc[0]
        specific_string_content = normalize_quotes(specific_string_content)
        try:
            dict_content = ast.literal_eval(specific_string_content)
            return dict_content
        except:
            claim_dict_eval_errors += 1
            print("ERROR: could not eval string as dict: " + specific_string_content)
            return {}
    else:
        return {}

In [12]:
label_assignment_prompt = """For each claim in article A, check for all claims in article B, 
whether the claim made in article B contradicts the claim made in article A (-1), 
reinforces the claim made inarticle A (1), or whether the two claims are unrelated (0). 

Adhere to the following output format:
{"comparisons": [
    {
        'claim_article_a': "This is a claim made in article a"
        'claim_article_b': "This is a claim made in article b"
        'relation': -1 
    },
    {
        'claim_article_a': "This is a claim made in article a"
        'claim_article_b': "This is another claim made in article b"
        'relation': 1
    },
    {
        'claim_article_a': "This is a claim made in article a"
        'claim_article_b': "This is yet another claim made in article b"
        'relation': 0
    }
]
}

Please consider the following:

• Your response must be directly the JSON, do not add any other text, such as
”Here is the output:” and similar.
• Make sure that the JSON is properly formatted, especially with correct types of braces and brackets
• Make sure to consider all of the possible claim pairs, which should add up to 5*5 = 25


Now do this task for the following input:
"""

In [13]:
def extract_claims(claim_dict):
    return [claim['claim'] for claim in claim_dict['claims']]

Prompting the model

In [14]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv() 

api_key = os.getenv("API_KEY") 
base_url = "https://chat-ai.academiccloud.de/v1"
model = "meta-llama-3.1-70b-instruct"  # Choose any available model
temperature = 0.01

# Start OpenAI client
client = OpenAI(
    api_key=api_key,
    base_url=base_url
    )

def process_prompt(prompt):
    # Get response
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "SYSTEM MESSAGE"},
            {"role": "user", "content": prompt}
        ],
        model=model,
        temperature=temperature
    )

    return chat_completion.choices[0].message.content

print(process_prompt("Hello!"))

Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?


In [15]:
import time
last_call_time = time.time()

def get_throttled_prompt_response(value):
    global last_call_time
    global response_errors
    elapsed_time = time.time() - last_call_time #dynamic buffer
    if elapsed_time < 8:
        time.sleep(8 - elapsed_time)
    try:
        last_call_time = time.time()
        return process_prompt(f"{label_assignment_prompt}{value}")
    except:
        response_errors += 1
        return "ERROR: " + str(value)

In [16]:
#can be used to retrieve domains later
def get_newsguard_dict(d, s):
    domains = pre_process_domains(d)
    try:
        domains_newsguard = ast.literal_eval(s)
    except:
        return {}

    try:
        ng_dict = {k:v for k, v in zip(domains, domains_newsguard)}
        return ng_dict
    except:
        return {}

In [17]:
search_result_df['list_domains']

0      newsweek.com,yahoo.com,azmirror.com,fr24news.c...
1      abc15.com,salon.com,cnn.com,politico.com,cbsne...
2      cnn.com,thehill.com,salon.com,washingtontimes....
3      thehill.com,recorder.maricopa.gov,washingtonpo...
4      apnews.com,detroitnews.com,cnn.com,apnews.com,...
                             ...                        
956    factcheck.org,yahoo.com,theintercept.com,polit...
957    googleadservices.com,scientificamerican.com,wh...
958    covid19.nih.gov,grants.nih.gov,grants.nih.gov,...
959    science.org,the-scientist.com,rollcall.com,van...
960    ',zerohedge.com,madisonarealymesupportgroup.co...
Name: list_domains, Length: 961, dtype: object

In [18]:
search_result_df['list_domains'] = search_result_df['list_domains'].str.replace(",", "', '")
search_result_df['list_domains']

C:\Users\danie\AppData\Local\Temp\ipykernel_17200\1022389342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_result_df['list_domains'] = search_result_df['list_domains'].str.replace(",", "', '")


0      newsweek.com', 'yahoo.com', 'azmirror.com', 'f...
1      abc15.com', 'salon.com', 'cnn.com', 'politico....
2      cnn.com', 'thehill.com', 'salon.com', 'washing...
3      thehill.com', 'recorder.maricopa.gov', 'washin...
4      apnews.com', 'detroitnews.com', 'cnn.com', 'ap...
                             ...                        
956    factcheck.org', 'yahoo.com', 'theintercept.com...
957    googleadservices.com', 'scientificamerican.com...
958    covid19.nih.gov', 'grants.nih.gov', 'grants.ni...
959    science.org', 'the-scientist.com', 'rollcall.c...
960    '', 'zerohedge.com', 'madisonarealymesupportgr...
Name: list_domains, Length: 961, dtype: object

In [19]:
search_result_df['newsguard_dict'] = search_result_df.apply(lambda row: get_newsguard_dict(row['list_domains'], row['list_scores']), axis=1)

ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string of domains to list
ERROR: could not convert string

C:\Users\danie\AppData\Local\Temp\ipykernel_17200\3065695327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_result_df['newsguard_dict'] = search_result_df.apply(lambda row: get_newsguard_dict(row['list_domains'], row['list_scores']), axis=1)


In [20]:
from urllib.parse import urlparse

def get_newsguard_score(raw_url, score_dict):
    global ng_dict_errors
    global ng_score_retrieval_errors
    if not score_dict:
        ng_dict_errors += 1
        return None
    try:
        domain = urlparse(raw_url).netloc #to get the the desired format
        return(score_dict['.'.join(domain.split('.')[-2:])])
    except:
        ng_score_retrieval_errors += 1
        return None
    

In [21]:
def get_claim_positions(claim_dict, relative=True):
    global claim_retrieval_errors
    positions = []
    try:
        for c in claim_dict['claims']:
            if relative == False:
                positions.append(c['position'])
            else:
                positions.append((c['position']/claim_dict['article_length']))
    except:
         claim_retrieval_errors += 1
         print("ERROR: could not extract positions")
    return positions

def get_claim_headline_info(claim_dict):
    global claim_headline_retrieval_errors
    h_info = []
    try:
        for c in claim_dict['claims']:
            h_info.append(c['headline'])
    except:
         claim_headline_retrieval_errors += 1
         print("ERROR: could not extract headline info")
    return h_info

In [22]:
def process_response(response, fm_url, serp_article, newsguard_dict, fm_dict):
    response_dict = ast.literal_eval(response)
    response_dict['fm_url'] = fm_url
    response_dict['fm_article_length'] = fm_dict['article_length']
    response_dict['serp_url'] = serp_article['url']
    response_dict['serp_position'] = serp_article['serp_position']
    response_dict['serp_article_length'] = serp_article['article_length']
    try:
        response_dict['serp_newsguard'] = get_newsguard_score(serp_article['url'], newsguard_dict)
    except:
        response_dict['serp_newsguard'] = None

    #get the relative positions of the claims
    positions_fake_article = get_claim_positions(fm_dict)
    positions_fake_article = [item for x in positions_fake_article for item in [x]*5]
    for comp, position in zip(response_dict['comparisons'], positions_fake_article):
        comp['fm_claim_position'] =  position
    positions_search_article = get_claim_positions(serp_article)
    positions_search_article = positions_search_article*5
    for comp, position in zip(response_dict['comparisons'], positions_search_article):
        comp['serp_claim_position'] =  position

    #get the absolute positions of the claims
    positions_fake_article = get_claim_positions(fm_dict, relative=False)
    positions_fake_article = [item for x in positions_fake_article for item in [x]*5]
    for comp, position in zip(response_dict['comparisons'], positions_fake_article):
        comp['fm_claim_position_abs'] =  position
    positions_search_article = get_claim_positions(serp_article, relative=False)
    positions_search_article = positions_search_article*5
    for comp, position in zip(response_dict['comparisons'], positions_search_article):
        comp['serp_claim_position_abs'] =  position 

    h_info_fake_article = get_claim_headline_info(fm_dict)
    h_info_fake_article = [item for x in h_info_fake_article for item in [x]*5]
    for comp, h_info in zip(response_dict['comparisons'], h_info_fake_article):
        comp['fm_claim_headline'] =  h_info

    h_info_serp_article = get_claim_headline_info(serp_article)
    h_info_serp_article = h_info_serp_article*5
    for comp, h_info in zip(response_dict['comparisons'], h_info_serp_article):
        comp['serp_claim_headline'] =  h_info
        
    return response_dict

In [23]:
#catch empty cases
def label_assignment(fm_url, serp_urls, newsguard_dict):
    label_list = []
    global fm_article_url_not_found
    global post_processing_error
    global serp_dict_empty_error

    fm_article_dict = get_claim_dict(fm_url, fm_articles_df) #get fm_claim_dict, this is also called in the get_serp_articles function, but that one also has some more details
    if not fm_article_dict:
        fm_article_url_not_found += 1
        return "ERROR: FM Article Claims could not be retrieved"
    fm_article_claims = extract_claims(fm_article_dict)

    serp_dict = get_serp_dict(serp_urls=serp_urls)
    if not serp_dict: #check for empty list
        serp_dict_empty_error += 1
        return None
    for serp_article in serp_dict:
        if not serp_article:
            label_list.append({})
            continue
        serp_claims = extract_claims(serp_article) #extract claims from the entire dict
        comparison_dict = {'claims_article_a': fm_article_claims, 'claims_article_b': serp_claims} #pass only the pure claims into the model
        response = get_throttled_prompt_response(comparison_dict)
        if "ERROR" in response:
            label_list.append({})
            continue
        try:
            processed_response = process_response(response, fm_url, serp_article, newsguard_dict, fm_article_dict)       
            label_list.append(processed_response)
        except:
            print("ERROR: Post-Processing Error")
            post_processing_error += 1
            # catch faulty responses
            label_list.append({})
    return label_list
    

#returns serp articles where a valid claim_dict could be retrieved

def get_serp_dict(serp_urls):
    global serp_article_not_found
    global successful_parse
    serp_articles = []  
    for i, serp_url in enumerate(serp_urls):
        try: 
            article = get_claim_dict(serp_url, claim_df)
            if isinstance(article, dict) and article:
                article['url'] = serp_url
                article['serp_position'] = i + 1
                serp_articles.append(article)
                successful_parse += 1
            else:
                serp_article_not_found += 1
                serp_articles.append({})
        except TypeError as e:
            print(f"TypeError for URL {serp_url}: {e}")
            continue
        except Exception as e:
            print(f"Unexpected error for URL {serp_url}: {e}")
    return serp_articles

In [ ]:
#change these variables to conduct over multiple sessions, 100 at a time, since we have up to 10 API calls for each entry (1 for each SERP article) -> 1000 API calls per session
current_start = 300 #Completed runs: 0-100; 100-200 (but 215 API response errors); 200-300;
current_end = current_start + 100

In [25]:
current_slice = search_result_df.iloc[current_start:current_end]
current_slice

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_85,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls,newsguard_dict
200,Day_10_3,R_XM6WITRk9ItxUVX,Day_10,True,1,6,3,20,0,FM,...,0.0,0.0,0.0,0.0,0.0,80.625000,"tasteofhome.com', 'tasteofhome.com', 'tasteofh...","75,70,95,82.5",(https://www.tasteofhome.com/article/food-shor...,{'tasteofhome.com': 82.5}
201,Day_10_3,R_3KoQtZqxlX0Pfa5,Day_10,Coul,0,4,3,45,0,FM,...,0.0,0.0,1.0,1.0,1.0,46.428571,"naturalnews.com', 'nytimes.com', 'nytimes.com'...","5,100,100,5,5,100,10",(https://www.naturalnews.com/2021-11-01-us-fac...,"{'naturalnews.com': 5, 'nytimes.com': 100, 'di..."
202,Day_10_3,R_NXwVIucKBLa4dP3,Day_10,Misl,0,2,2,30,0,FM,...,0.0,0.0,0.0,0.0,0.0,97.812500,"theguardian.com', 'theguardian.com', 'un.org',...","100,100,100,100,82.5,100,100,100",(https://www.theguardian.com/global-developmen...,"{'theguardian.com': 100, 'un.org': 100, 'relie..."
203,Day_10_3,R_2sGvyvdHbUmZ02Z,Day_10,Misl,0,6,3,32,0,FM,...,0.0,0.0,0.0,0.0,0.0,95.000000,"googleadservices.com', 'extension.okstate.edu'...","100,100,92.5,100,82.5","(https://www.googleadservices.com/, https://ex...","{'googleadservices.com': 100, 'extension.oksta..."
204,Day_10_3,R_QmEgpaHZFkFLQlP,Day_10,Misl,0,1,1,28,0,FM,...,1.0,0.0,0.0,0.0,0.0,93.125000,"cnbc.com', 'abc7chicago.com', 'whitehouse.gov'...","87.5,100,87.5,87.5,100,87.5,95,100",(https://www.cnbc.com/2021/10/29/biden-to-disc...,"{'cnbc.com': 87.5, 'abc7chicago.com': 100, 'wh..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Day_11_1,R_3CU6zJcLSdm2yG0,Day_11,Misl,0,1,2,28,0,FM,...,0.0,0.0,0.0,0.0,0.0,80.000000,"realclearpolitics.com', 'realclearpolitics.com...","80,80,80",(https://www.realclearpolitics.com/video/2021/...,{'realclearpolitics.com': 80}
296,Day_11_1,R_2BrFFcpSaQ2qL6C,Day_11,Misl,0,3,2,35,0,FM,...,0.0,0.0,0.0,0.0,1.0,87.750000,"youtube.com', 'skynews.com.au', 'youtube.com',...","100,82,100,69.5,87.5,87.5","(https://www.youtube.com/watch?v=4Ht3MmBZHq4, ...","{'youtube.com': 69.5, 'skynews.com.au': 82, 'n..."
297,Day_11_1,R_3CU6zJcLSdm2yG0,Day_11,Misl,0,1,2,28,0,FM,...,1.0,1.0,0.0,0.0,0.0,100.000000,"skynews.com.au', 'skynews.com.au', 'skynews.co...",100,(https://www.skynews.com.au/opinion/alan-jones...,{}
298,Day_11_1,R_2q4VcH13UlJ2MF6,Day_11,Coul,0,4,3,31,0,FM,...,0.0,0.0,0.0,0.0,0.0,80.000000,"realclearpolitics.com', 'skynews.com.au', 'sky...",80,(https://www.realclearpolitics.com/video/2021/...,{}


In [26]:
reset_error_count()
current_slice['Labels'] = current_slice.apply(lambda row: label_assignment(row['URL'], row['all_clean_urls'], row['newsguard_dict']), axis=1)
create_log(f'./logs/error_log{current_start}_{current_end}.txt')
current_slice.to_csv(f"{data_folder}/label_data_{current_start}_{current_end}.csv", sep=";")
current_slice['Labels'].to_json(f"{data_folder}/label_data_{current_start}_{current_end}.json")

ERROR: could not eval string as dict: {
"article_title": "Coronavirus pandemic 'will cause famine of biblical proportions'",
"url": "https://www.theguardian.com/global-development/2020/apr/21/coronavirus-pandemic-will-cause-famine-of-biblical-proportions-un-warns",
"article_length": 37,
"claims": [
{
"claim": "The world is facing widespread famine "of biblical proportions" because of the coronavirus pandemic, the chief of the UN's food relief agency has warned.",
"headline": 1,
"position": 1
},
{
"claim": "More than 30 countries in the developing world could experience widespread famine, and in 10 of those countries there are already more than 1 million people on the brink of starvation.",
"headline": 0,
"position": 2
},
{
"claim": "At least 265 million people are being pushed to the brink of starvation by the Covid-19 crisis, double the number under threat before the pandemic.",
"headline": 0,
"position": 19
},
{
"claim": "The World Food Programme needs about $2bn (£1.6bn) of aid to g

C:\Users\danie\AppData\Local\Temp\ipykernel_17200\628564922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_slice['Labels'] = current_slice.apply(lambda row: label_assignment(row['URL'], row['all_clean_urls'], row['newsguard_dict']), axis=1)


Post-Processing Results

In [27]:
# influence = positions_search_article - positions_fake_article
# print(positions_fake_article[raw_labels != 0])
# print(positions_search_article[raw_labels != 0])
# influence = influence[raw_labels != 0]
# influence

In [28]:
# prelim_score = np.sum(influence)/len(influence)
# prelim_score